In [1]:
#General packages
import numpy as np
from My_Jampy import JAM
import emcee

#MPI
from schwimmbad import MPIPool

#Constants and usefull packages
from astropy.cosmology import Planck15 as cosmo
from astropy.constants import G, M_sun, c
import astropy.units as u

#Autolens Model packages
import autolens as al
import autolens.plot as aplt

#Combined Model package
import CombinedModel

#Useful constants
metre2Mpc = (1*u.m).to(u.Mpc)/u.m           #Constant factor to convert metre to Mpc.
kg2Msun = (1*u.kg/M_sun)*u.solMass/u.kg     #Constant factor to convert kg to Msun

G_Mpc = G*(metre2Mpc)**3/kg2Msun            #Gravitational constant in Mpc³/(Msun s²)
c_Mpc = c*metre2Mpc                         #Speed of light in Mpc/s


#Dataset path
data_folder = "/home/carlos/Documents/GitHub/Master-Degree/Autolens_tests/autolens_workspace/Test_3/Simulation_data/"

In [2]:
#Reading MGE inputs
            #attention to units
surf_lum, sigma_lum, qobs_lum = np.loadtxt("JAM_Input.txt", unpack=True)          #MGE decomposition
surf_dm, sigma_dm , qobs_dm   = np.loadtxt("SDP81_pseudo-DM.txt", unpack=True)    #DM component
norm_psf, sigma_psf           = np.loadtxt("MUSE_Psf_model.txt", unpack=True)     #PSF
xbin, ybin, vrms              = np.loadtxt("vrms_data.txt", unpack=True)          #Vrms data

muse_pixsize = 0.2                            #Muse pixel size [arcsec/px]

z_lens   = 0.299                                    #Lens redshifth
z_source = 3.100                                    #Source redshift

#Angular diameter distances
D_l = cosmo.angular_diameter_distance(z_lens)                   #Lens              
D_s = cosmo.angular_diameter_distance(z_source)                 #Source
D_ls = cosmo.angular_diameter_distance_z1z2(z_lens, z_source)   #Lens to source

## Models inicialization

"""
    To inicialize the model, we set some random values for the parameters. But it's only necessary for initialize the model. During the non-linear search, this values will be updated constantly until the best fit.
"""  


   #This quantities are our unknown parameters
inc       = 75                              #Inclination [deg]
mbh       = 1e10                            #Mass of black hole [M_sun]
beta      = np.full_like(surf_lum, 0.3)     #Anisotropy
ml        = 10                              #Mass to light ratio [M_sun/L_sun]
mag_shear = 0.01                            #Shear magnitude
phi_shear = 100.0                             #Shear angle
rho_s     = 1e10                            #dark matter intensity
qdm       = np.full_like(qobs_dm, 0.5)      #dark matter axial ratio
gamma     = 1.0                             #Gamma

#--------------------------------------------------------------------------------------------------#
# JAMPY MODEL

Jam_model = JAM(ybin=ybin, xbin=xbin, inc=inc, distance=D_l.value, mbh=mbh, beta=beta, rms=vrms,
               normpsf=norm_psf, sigmapsf=sigma_psf*muse_pixsize, pixsize=muse_pixsize)

    #Add Luminosity component
Jam_model.luminosity_component(surf_lum=surf_lum, sigma_lum=sigma_lum,
                                qobs_lum=qobs_lum, ml=ml)
    #Add DM component
Jam_model.DM_component(surf_dm=rho_s * surf_dm, sigma_dm=sigma_dm, qobs_dm=qdm)


#--------------------------------------------------------------------------------------------------#
# PYAUTOLENS MODEL

imaging = al.Imaging.from_fits(
    image_path=f"{data_folder}/arcs_simulation.fits",
    noise_map_path=f"{data_folder}/noise_simulation.fits",
    psf_path=f"{data_folder}/psf_simulation.fits",
    pixel_scales=0.1,
)

mask = al.Mask.circular_annular(centre=(0.0, -0.2), inner_radius=1., outer_radius=2.3,
                          pixel_scales=imaging.pixel_scales, shape_2d=imaging.shape_2d) #Create a mask

masked_image = al.MaskedImaging(imaging=imaging, mask=mask, inversion_uses_border=True) #Masked image

mass_profile = al.mp.MGE()

    #Components
mass_profile.MGE_comps(z_l=z_lens, z_s=z_source, 
                   surf_lum=surf_lum, sigma_lum=sigma_lum, qobs_lum=qobs_lum, ml=ml,
                   mbh=mbh, surf_dm =rho_s * surf_dm, sigma_dm=sigma_dm, qobs_dm=qdm)

#--------------------------------------------------------------------------------------------------#
# COMBINED MODEL

    #Just remembering, by default the model does not include dark matter.
model = CombinedModel.Models(Jampy_model=Jam_model, mass_profile=mass_profile,
                             masked_imaging=masked_image, quiet=True)

model.mass_to_light(ml_kind='scalar')               #Setting gradient ML
model.beta(beta_kind='scalar')                      #Seting vector anisotropy
model.has_DM(a=True,filename="SDP81_pseudo-DM.txt") #Setting Dark matter component
#--------------------------------------------------------------------------------------------------#

In [3]:
def call_model(x0):
    #print(x0)
    value =  model(x0)
    #print(value)
    if np.isfinite(value):
        return -value
    else:
        return 1e200

In [4]:
from scipy.optimize import dual_annealing

In [19]:
#In order: ML, beta, inc, log_mbh, log_rho_s, qDM, mag_shear, phi_shear, gamma
bounds = [(1, 12), (-2, 2), (40, 90), (6, 12), (6, 12), (0.2, 1), (0, 0.1), (0, 179), (0.8, 1.2)]

In [22]:
result = dual_annealing(call_model, bounds)

[ 1.09635869e+01 -3.22214925e-01  8.37357985e+01  6.59984645e+00
  7.09920855e+00  3.53974049e-01  3.29255049e-02  9.43485245e+01
  1.04702420e+00]
-inf
[3.48003985e+00 1.97286453e+00 7.72522514e+01 6.89492590e+00
 8.61566146e+00 6.70426956e-01 2.80049617e-02 1.20864977e+02
 9.63477111e-01]
-inf
[9.40999979e+00 1.12419793e+00 5.51822114e+01 8.11181248e+00
 8.61117459e+00 2.00386905e-01 4.48915355e-02 1.01794937e+02
 9.58990229e-01]
-inf
[3.96020098e+00 6.74399115e-01 5.67324126e+01 8.41976324e+00
 8.91912535e+00 5.50588089e-01 2.95395458e-02 1.53345139e+02
 8.66941003e-01]
-inf
[4.72392577e+00 1.50920634e+00 8.84961374e+01 9.36018528e+00
 9.85954739e+00 3.14312882e-01 6.43467764e-02 7.41799458e+01
 1.00736305e+00]
-79325.84902066363
[ 9.70524683e+00 -5.09472601e-01  4.45689279e+01  7.34150634e+00
  9.93233795e+00  6.95633937e-01  3.71373427e-02  2.02527364e+01
  1.08015360e+00]
-inf
[4.35764018e+00 1.42920755e-01 7.91298518e+01 7.99389970e+00
 9.68287352e+00 5.48027297e-01 9.80611917e-

-32666.44928623331
[ 9.52463693 -1.3073569  81.11333252  6.29463352 10.84190748  0.74402929
  0.08154194 66.9290029   0.87367318]
-inf
[ 1.78610864e+00 -3.63138265e-01  8.32768671e+01  8.14091504e+00
  6.68818900e+00  7.24841681e-01  4.50765257e-02  1.07190475e+02
  1.01789183e+00]
-30025.777087413855
[ 5.32868561 -1.63782463 82.91738119 11.96416578  8.32870309  0.25015531
  0.08559062 34.47983811  1.1604688 ]
-inf
[7.28087962e+00 1.31632723e-01 8.88695752e+01 1.17336231e+01
 1.02808971e+01 4.19612664e-01 3.77846282e-02 4.21921191e+01
 8.19536319e-01]
-inf
[1.03251036e+01 1.07427740e+00 8.38122199e+01 1.17778471e+01
 1.03251211e+01 5.62257349e-01 8.20086287e-02 5.06278903e+01
 9.53825505e-01]
-inf
[ 8.09456675e+00 -1.78192310e+00  4.96832623e+01  9.82006731e+00
  1.10945842e+01  7.65988935e-01  5.14717517e-02  1.76771690e+02
  1.12634996e+00]
-inf
[ 9.89095934e+00 -1.05229248e+00  6.96856500e+01  1.05496979e+01
  9.09697190e+00  8.35687465e-01  5.38594258e-02  1.54501320e+02
  1.128737

-17904.347812763262
[9.89095934e+00 1.20279890e+00 6.96856500e+01 1.05496979e+01
 9.61821316e+00 8.35687465e-01 5.38594258e-02 1.44954122e+02
 1.12873762e+00]
-187625.44754379836
[ 9.89095934e+00 -1.05229248e+00  5.33804883e+01  1.05496979e+01
  9.61821316e+00  8.35687465e-01  5.38594258e-02  1.44954122e+02
  1.12873762e+00]
-inf
[ 9.89095934e+00 -1.05229248e+00  6.96856500e+01  1.08475772e+01
  9.61821316e+00  8.35687465e-01  5.38594258e-02  1.44954122e+02
  1.12873762e+00]
-14951.045242751412
[ 9.89095934e+00 -1.05229248e+00  6.96856500e+01  1.08475772e+01
  6.24782302e+00  8.35687465e-01  5.38594258e-02  1.44954122e+02
  1.12873762e+00]
-14067.038237309058
[ 9.89095934e+00 -1.05229248e+00  6.96856500e+01  1.08475772e+01
  6.24782302e+00  5.88671389e-01  5.38594258e-02  1.44954122e+02
  1.12873762e+00]
-14135.822286176817
[ 9.89095934e+00 -1.05229248e+00  6.96856500e+01  1.08475772e+01
  6.24782302e+00  8.35687465e-01  3.41968101e-03  1.44954122e+02
  1.12873762e+00]
-13998.579174991

-13419.271414449866
[ 9.89095934e+00 -9.34029847e-01  6.96856500e+01  1.08475772e+01
  6.24782302e+00  7.40955113e-01  3.41968101e-03  4.14223458e+01
  1.12873762e+00]
-13393.418466548179
[ 9.89095934e+00 -9.34029847e-01  6.96856500e+01  1.08475772e+01
  6.24782302e+00  8.35687465e-01  6.94954552e-02  4.14223458e+01
  1.12873762e+00]
-17715.8034787738
[ 9.89095934e+00 -9.34029847e-01  6.96856500e+01  1.08475772e+01
  6.24782302e+00  8.35687465e-01  3.41968101e-03  1.56217492e+02
  1.12873762e+00]
-13470.530686325936
[ 9.89095934e+00 -9.34029847e-01  6.96856500e+01  1.08475772e+01
  6.24782302e+00  8.35687465e-01  3.41968101e-03  4.14223458e+01
  9.06977194e-01]
-30794.56748281417
[ 9.89095934e+00 -9.34029847e-01  6.96856500e+01  1.08475772e+01
  6.24782302e+00  8.35687465e-01  3.41968101e-03  4.14223458e+01
  1.12873762e+00]
-13360.757620462435
[ 9.89095935e+00 -9.34029847e-01  6.96856500e+01  1.08475772e+01
  6.24782302e+00  8.35687465e-01  3.41968101e-03  4.14223458e+01
  1.12873762e

-13605.470300624242
[ 9.89095934e+00 -9.34029847e-01  6.96856500e+01  1.08475772e+01
  6.24782302e+00  8.35687465e-01  3.41968101e-03  4.14223458e+01
  8.25668032e-01]
-69390.12593410176
[ 4.07418097e+00 -9.43179268e-01  5.63516642e+01  1.09955730e+01
  6.19106983e+00  3.86467167e-01  4.61465124e-02  1.02570342e+02
  8.10450023e-01]
-inf
[ 8.68319746  0.31221227 60.87224673  7.92490457  9.49406514  0.48192958
  0.0772181  44.29575889  1.12063411]
-inf
[ 1.60168323e+00 -1.22330596e+00  4.06106316e+01  7.77255880e+00
  9.23956547e+00  5.60669101e-01  9.60500963e-03  1.03347327e+02
  8.39461489e-01]
-inf
[ 1.01730523e+01 -5.97103597e-01  6.30439823e+01  6.52514646e+00
  1.15299160e+01  8.10811905e-01  3.41828517e-02  1.04063496e+02
  8.10830568e-01]
-inf
[ 1.10689268e+01 -1.75606242e+00  8.13730651e+01  1.07079983e+01
  8.14644187e+00  9.23102535e-01  3.14099970e-02  1.05292193e+02
  8.16152689e-01]
-35415.74378232467
[2.19490051e+00 1.84587399e-01 5.29895912e+01 8.37243736e+00
 7.6714009

...

-64023.29249993717
[ 7.61156727  0.75452542 71.79717211  8.73609771  7.50303165  0.895737
  0.09632567 95.75552893  1.00814016]
-89297.89482659986
[8.47818784e+00 7.74076281e-01 7.18283566e+01 7.96560611e+00
 8.47630897e+00 4.98954906e-01 2.10133723e-02 9.59719320e+01
 1.03780859e+00]
-117897.69339077099
[ 8.31848777e+00 -7.96657962e-01  7.17937632e+01  8.73973426e+00
  8.11569249e+00  7.99022200e-01  8.25960123e-02  9.85938096e+01
  1.01285039e+00]
-53682.20180387056
[ 7.30050209e+00 -8.09190527e-01  7.19000207e+01  8.74394314e+00
  8.55240402e+00  3.60386670e-01  6.96835424e-02  9.79815802e+01
  1.13346911e+00]
-18755.02158993315
[ 3.75067510e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-20713.391055110314
[ 7.54494365e+00 -4.03849365e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-6167.730619184054
[ 7.54494365e+00 -8.11076

-6360.3495194222605
[ 8.97533644e+00 -7.95776660e-01  7.14820199e+01  8.73955655e+00
  1.16119677e+01  8.96527535e-01  3.12173704e-02  9.57367081e+01
  1.15148417e+00]
-66769.44088138436
[ 1.02360704e+01 -8.02492115e-01  5.87320819e+01  8.58110602e+00
  8.54674446e+00  9.44207155e-01  2.03602176e-02  7.54327748e+01
  9.65088361e-01]
-62203.95360074006
[ 8.27415598e+00 -8.10741022e-01  7.17923109e+01  1.07579810e+01
  6.54253567e+00  9.76422611e-01  5.84492434e-03  9.57574005e+01
  1.05006887e+00]
-101110.55408924814
[ 7.46951360e+00 -8.09176263e-01  7.17968333e+01  1.14549436e+01
  8.39996790e+00  9.50554023e-01  1.81504780e-02  9.57731457e+01
  1.18057069e+00]
-inf
[6.48141154e+00 1.61716219e+00 7.22447705e+01 7.16400049e+00
 8.62388819e+00 3.22869916e-01 5.82141535e-02 9.35297168e+01
 8.76232195e-01]
-126606.13053672586
[ 6.73226271e+00 -7.60117978e-01  7.17958549e+01  8.65800869e+00
  8.54011608e+00  9.73582183e-01  2.38296034e-02  1.03640984e+02
  1.12885188e+00]
-27059.9236814804


-17998.857425055052
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57225696e+01
  1.03801411e+00]
-5929.06005256057
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  8.70976251e-01]
-35360.215689641576
[7.54368614e+00 4.52040562e-01 7.19613720e+01 8.74381729e+00
 8.16417705e+00 8.75773108e-01 2.31875341e-02 3.13918979e+01
 1.03960379e+00]
-62041.65588227647
[ 7.67487240e+00 -7.37715750e-01  4.55607513e+01  8.73709907e+00
  8.68995076e+00  3.75058155e-01  2.05675227e-02  9.57509408e+01
  9.97877870e-01]
-inf
[ 7.54774118e+00 -8.10071577e-01  7.18163024e+01  8.27021309e+00
  8.54910651e+00  9.66177039e-01  3.08903142e-02  9.57558915e+01
  1.01324198e+00]
-7464.564366192906
[ 7.54602921e+00 -7.55796226e-01  7.17864967e+01  8.74171940e+00
  9.74247052e+00  9.61301105e-01  7.48884408e-02  1.64996132e+02
  9.23502745e-01]
-46017.53835779601
[

-5931.174001407517
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.82424326e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-6035.385784345894
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.55083934e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-5931.734585688266
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  2.10475851e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-inf
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.50544760e-02  9.57510708e+01
  1.03801411e+00]
-6474.192512108254
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.54416448e+01
  1.03801411e+00]
-5978.40025362682
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.02895126e+00]
-6279.2792837

-28625.728065696858
[ 1.00020666e+01 -8.03630701e-01  7.17966874e+01  8.31882097e+00
  9.61822385e+00  3.18066931e-01  6.14600451e-02  9.51692229e+01
  1.13186439e+00]
-35178.46935703831
[ 7.52608081e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-6085.450936525551
[ 7.54494365e+00 -1.05828182e+00  7.17961899e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-5998.52689517569
[ 7.54494365e+00 -8.11076878e-01  7.17960409e+01  8.73838450e+00
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-5925.230845643475
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  1.12624523e+01
  8.54819992e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-inf
[ 7.54494365e+00 -8.11076878e-01  7.17961899e+01  8.73838450e+00
  8.56197505e+00  9.62965243e-01  2.16824764e-02  9.57510708e+01
  1.03801411e+00]
-5933.123631

In [23]:
print(result)

     fun: 5920.161295632374
 message: ['Maximum number of iteration reached']
    nfev: 20011
    nhev: 0
     nit: 1000
    njev: 201
  status: 0
 success: True
       x: array([ 7.54494365e+00, -8.11076878e-01,  7.17961899e+01,  8.73838450e+00,
        8.54819992e+00,  9.62965243e-01,  2.16824764e-02,  9.57510708e+01,
        1.03801411e+00])
